In [ ]:
import openpyxl
import pandas as pd
import json
#import requests
import os
import time
import re
import openai
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
import os

load_dotenv("template.env")

# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
    raise ValueError("La clave de API no está configurada en el archivo .env")
    
client = OpenAI() #creando un objeto de la clase

In [ ]:
dataset_path = "MMLU_completo.xlsx"

df = pd.read_excel(dataset_path)
#df = df.sample(20) #QUITAR ESTA LÍNEA PARA PROCESAR TODO EL DATASET
df = df.sample(5) 
df.head()

In [ ]:
categorize_system_prompt1 = '''
Your goal is to evaluate multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the evaluation of the question.

For each row evaluate the question.

The output JSON objects should be in the following format:

{
    answer: string, // The selected option key for the question, limited to 'A', 'B', 'C', or 'D'
}

Keep the JSON format in the answer with '{' and '}'.
'''

In [ ]:
categorize_system_prompt ='''
Your goal is to paraphrase multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the reworded question.

For each row paraphrase the question while maintaining the original meaning and paraphrase the options as well. Ensure the paraphrase is at least 500 characters long between options and question.

The output JSON objects should be in the following format:

{paraphrased_question: string, option_a: string, option_b: string,option_c: string,option_d: string,}

Ensure that the question is presented differently but conveys the same idea. Keep the JSON format in the output with '{' and '}'.
'''

In [ ]:
tasks = []

for index, row in df.iterrows():
    
    # Extraer las columnas necesarias del DataFrame
    instruction = row['instruction']
    option_a = row['option_a']
    option_b = row['option_b']
    option_c = row['option_c']
    option_d = row['option_d']
    
    # Formar el JSON con las columnas extraídas
    description = json.dumps({
        "instruction": instruction,
        "options": {
            "A": option_a,
            "B": option_b,
            "C": option_c,
            "D": option_d
        }
    })
    
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # Esto es lo que tendrías en tu llamada a la API de Chat Completions
            "model": "gpt-4o-mini",
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": categorize_system_prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }
    
    tasks.append(task)

In [ ]:
for i in range(0,5):
	print(df.values[i])

In [ ]:
# Creating the file

file_name = "batch_tasks_mmlu.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [ ]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [ ]:
print(batch_file)

In [ ]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [ ]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

In [ ]:
#PARA CANCELAR UN BATCH
# # ID del batch que deseas cancelar
# batch_id = batch_job.id

# # URL para cancelar el batch
# url = f"https://api.openai.com/v1/batches/{batch_id}/cancel"

# # Encabezados de la solicitud
# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json",
# }

# # Realiza la solicitud POST para cancelar el batch
# response = requests.post(url, headers=headers)

# # Maneja la respuesta
# if response.status_code == 200:
#     data = response.json()
#     print("Batch cancelado exitosamente.")
#     print("Detalles del Batch:")
#     print(data)
# else:
#     print(f"Error al cancelar el batch: {response.status_code}")
#     print(response.text)

In [ ]:
# PARA EXTRAER LOS RESULTADOS
result_file_id = batch_job.output_file_id
print(batch_job.status)

In [ ]:
result = client.files.content(result_file_id).content

In [ ]:
result_file_name = "batch_job_results_mmlu.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [ ]:
jsonl_file = 'batch_job_results_mmlu.jsonl'
data = []
with open(jsonl_file, 'r') as file:
    for line in file:
        new_line = json.loads(line)
        data.append(new_line)
        #print(new_line)
        
test_1 = data[0]
test_1 = test_1["response"]["body"]["choices"][0]["message"]["content"]
test_1 = json.loads(test_1)
instruct = test_1["paraphrased_question"]
op_a = test_1["option_a"]
op_b = test_1["option_b"]
op_c = test_1["option_c"]
op_d = test_1["option_d"]
print(instruct)
print(op_a)
print(op_b)
print(op_c)
print(op_d)


